In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import glob #read multiple files
import warnings
warnings.filterwarnings("ignore")
#display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import master_functions as mf

# I. Import and transoform option data

In [2]:
option_data = mf.import_data("option_data")
# make a copy of the original data

merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2016.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2020.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2017.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2011.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2012.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2013.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2014.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Option_data/delta_hedging_ATM_c_m30_all_2018.xlsx
merge /Users/edgarschmidt/Deskto

In [70]:
delta_test = option_data.head(5)
delta_test = mf.transform_strikeprice(delta_test)
delta_test = mf.calculate_optionprice_C(delta_test)
delta_test_returns = mf.calculate_discrete_delta_hedged_gains(delta_test)

In [73]:
delta_test

,ID,SecurityID,Datum,Symbol,Strike,Expiration,BestBid,BestOffer,Delta,OptionID,ClosePrice,date_diff,interest_rate,minus30,maturity,Optionprice_C,increment_2_part,increment_3_part
0,1,101149,2016-01-04,A 160219C37500,37.5,2016-02-19,3.65,4.00,0.782366,108058735,40.69,1,0.383999,2016-01-20,2016-02-19,3.82,-0.109531,-0.000295
1,2,101149,2016-01-05,A 160219C37500,37.5,2016-02-19,3.55,4.30,0.748533,108058735,40.55,1,0.382584,2016-01-20,2016-02-19,3.92,0.134736,-0.000277
2,3,101149,2016-01-06,A 160219C37500,37.5,2016-02-19,3.45,3.95,0.810864,108058735,40.73,1,0.382807,2016-01-20,2016-02-19,3.70,-1.402795,-0.000308
3,4,101149,2016-01-07,A 160219C37500,37.5,2016-02-19,2.55,2.87,0.651450,108058735,39.00,1,0.379657,2016-01-20,2016-02-19,2.71,-0.267094,-0.000236
4,5,101149,2016-01-08,A 160219C37500,37.5,2016-02-19,2.27,3.25,0.609088,108058735,38.59,3,0.383957,2016-01-20,2016-02-19,2.76,NaN,-0.000655


In [72]:
delta_test_returns

,OptionID,SecurityID,Strike,maturity,delta_gain
0,108058735,101149,37.5,2016-02-19,0.020934


In [3]:
help(mf.transform_strikeprice)
option_data_copy = pd.DataFrame(option_data.copy())
option_data_copy = mf.transform_strikeprice(option_data_copy)

Help on function transform_strikeprice in module master_functions:

transform_strikeprice(option_data)
    The original data source has exported a wrong format for the strike price. 
    This function transforms the strike price into the correct format.



In [4]:
help(mf.calculate_optionprice_C)
option_data_copy = mf.calculate_optionprice_C(option_data_copy)

Help on function calculate_optionprice_C in module master_functions:

calculate_optionprice_C(option_data)
    Later, in the delta-gain-hedge calculation, we need the option price C. 
    This function calculates the option price C as the middle price of best bid and best offer.



In [5]:
help(mf.filter_option_data_30_days_before_maturity)
option_data_copy = mf.filter_option_data_30_days_before_maturity(option_data_copy)

Help on function filter_option_data_30_days_before_maturity in module master_functions:

filter_option_data_30_days_before_maturity(option_data)
    We are only interested in the observations 30 days before maturity. 
    The other observations are not relevant for our analysis and are therefore deleted.

Shape before transformation:  (9080229, 16)
Shape after transformation:  (5961046, 16)


In [6]:
help(mf.delete_options_with_more_than_x_missings)
option_data_copy = mf.delete_options_with_more_than_x_missings(option_data_copy, 6)

Help on function delete_options_with_more_than_x_missings in module master_functions:

delete_options_with_more_than_x_missings(option_data, x)
    Missing Delta values are indicated with a value of -99,x. This function deletes all options that have more than x missing values.

Number of deleted options:  25549


In [7]:
help(mf.replace_missings)
option_data_copy = mf.replace_missings(option_data_copy)

Help on function replace_missings in module master_functions:

replace_missings(option_data)
    If we have less then 7 missing values, we can replace the missing values with the value of the previous day. 
    If there is no previous day, then we are using the next day.



In [8]:
option_data_copy.head(5)

,ID,SecurityID,Datum,Symbol,Strike,Expiration,BestBid,BestOffer,Delta,OptionID,ClosePrice,date_diff,interest_rate,minus30,maturity,Optionprice_C
11,12,101149,2016-01-20,A 160219C37500,37.5,2016-02-19,1.29,1.44,0.495310,108058735,37.26,1,0.382527,2016-01-20,2016-02-19,1.36
12,13,101149,2016-01-21,A 160219C37500,37.5,2016-02-19,1.30,1.37,0.491530,108058735,37.23,1,0.381428,2016-01-20,2016-02-19,1.34
13,14,101149,2016-01-22,A 160219C37500,37.5,2016-02-19,1.51,1.66,0.576211,108058735,37.98,3,0.388625,2016-01-20,2016-02-19,1.58
14,15,101149,2016-01-25,A 160219C37500,37.5,2016-02-19,1.30,1.39,0.532060,108058735,37.61,1,0.381930,2016-01-20,2016-02-19,1.34
15,16,101149,2016-01-26,A 160219C37500,37.5,2016-02-19,1.21,1.32,0.519331,108058735,37.51,1,0.385089,2016-01-20,2016-02-19,1.27


# II. Calculate Delta-Hedged-Gains

In [9]:
help(mf.calculate_discrete_delta_hedged_gains)
delta_gain_returns = mf.calculate_discrete_delta_hedged_gains(option_data_copy)

Help on function calculate_discrete_delta_hedged_gains in module master_functions:

calculate_discrete_delta_hedged_gains(option_data)
    Calculate Delta-Hedged-Gain-Return for each OptionID in the dataset.
    Return all results in a new table called "delta_gain_table".
    
    // This implementation uses vectorized operations, which can significantly speed up the calculation 15min -> 1min



In [10]:
delta_gain_returns.head()

,OptionID,SecurityID,Strike,maturity,delta_gain
0,34416330,102456,126.0,2011-06-17,0.003814
1,34615388,108226,40.0,2012-01-20,-0.029430
2,34615427,101795,30.0,2012-01-20,-0.028304
3,34615534,107532,35.0,2012-01-20,-0.004845
4,34615564,112487,50.0,2012-01-20,-0.012296


# III. Calculate Idiosyncratic Stock Volatility

In [11]:
stock_data = mf.import_data("stock_data")

merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_140000_06_2012_2014.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_112000_2018_2023.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_105000_2014_2016.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_140000_2018_2023.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_105000_2017_2023.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_rest_2000_2014.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securities_112000_08_2012_2014.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock_data/securties_rest_2015_10_2017.xlsx
merge /Users/edgarschmidt/Desktop/Masterarbeit/Datasource/Excel_Export/Stock

In [12]:
final_dataset = delta_gain_returns.sort_values(by=['SecurityID','maturity'], ascending=[True,True])

In [13]:
help(mf.calculate_volatilitys)
final_dataset = mf.calculate_volatilitys(final_dataset,stock_data)

Help on function calculate_volatilitys in module master_functions:

calculate_volatilitys(option_data, stock_data)
    Calculate volatilitys for each OptionID in the dataset.
    Return all results in a new table called "volatility_table".

Dimension after transofrmation:  (2509, 6269)
Dimension: (11323542, 4)
Dimension after dropping duplicates:  (2509, 4)


In [14]:
final_dataset.head()

,OptionID,SecurityID,Strike,maturity,delta_gain,maturity_minus_60_days,maturity_minus_30_days,security_variance,security_std_dev,idiosyncratic_volatility,systematic_volatility,systematic_part_percent,stock_data_available
210084,129252242,5139,25.0,2019-07-19,0.046667,2019-05-20,2019-06-19,0.001078,0.076835,0.000174,0.000904,83.84,False
210479,129418344,5139,25.0,2019-08-16,-0.031819,2019-06-17,2019-07-17,0.000672,0.060659,0.000109,0.000563,83.79,False
214650,130131273,5139,25.0,2019-10-18,-0.000958,2019-08-19,2019-09-18,0.001352,0.086066,0.000207,0.001146,84.72,False
216823,130589826,5139,25.0,2019-11-15,0.109616,2019-09-16,2019-10-16,0.000711,0.062420,0.000090,0.000621,87.31,False
222736,131735135,5139,30.0,2020-02-21,-0.055966,2019-12-23,2020-01-22,0.000636,0.059014,0.000163,0.000473,74.36,False


In [15]:
final_dataset = final_dataset.dropna()

# IV. Fama-MacBeth-Regression

In [16]:
help(mf.descriptive_statistics)
mf.descriptive_statistics(final_dataset)

Help on function descriptive_statistics in module master_functions:

descriptive_statistics(option_data)
    This function calculates the descriptive statistics for the option data.



,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,220533.0,-0.000955,0.067076,-4.659014e+00,-0.047301,-0.021838,-0.004011,0.014595,0.045340,4.668170
security_variance,220533.0,0.004428,0.034340,5.548191e-08,0.000450,0.000876,0.001904,0.004282,0.008890,11.827467
security_std_dev,220533.0,0.124233,0.093915,5.512594e-04,0.049619,0.069280,0.102113,0.153147,0.220659,8.048708
idiosyncratic_volatility,220533.0,0.000668,0.006135,6.917966e-11,0.000047,0.000101,0.000241,0.000595,0.001336,2.257320
systematic_volatility,220533.0,0.003760,0.028392,4.728263e-08,0.000391,0.000760,0.001645,0.003675,0.007593,9.570148
systematic_part_percent,220533.0,86.357239,4.738871,7.261000e+01,81.160000,82.360000,85.280000,89.400000,93.410000,100.000000


In [17]:
help(mf.final_regression)
mf.final_regression(final_dataset,True)

Help on function final_regression in module master_functions:

final_regression(option_data, constant)
    This function calculates the regression of the delta-gain-hedge (dependend variable) on the idiosyncratic volatility and the systematic volatility (independent variables).

                            OLS Regression Results                            
Dep. Variable:             delta_gain   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     120.9
Date:                Fri, 18 Aug 2023   Prob (F-statistic):           3.32e-53
Time:                        07:20:30   Log-Likelihood:             2.8306e+05
No. Observations:              220533   AIC:                        -5.661e+05
Df Residuals:                  220530   BIC:                        -5.661e+05
Df Model:                           2                                         
Covarianc

,0
const,-0.000655
idiosyncratic_volatility,0.048855
systematic_volatility,-0.088301


To be discussed: 
Paper says: 
- "delta-hedged equity option return decreases monotonically with an increase in the idiosyncratic volatility of the underlying stock" p.1
- "the coefficient on idiosyncratic volatility remains negative and significant" p.2
- "Total volatility (VOL) is the standard deviation of daily stock returns" Table 1
- "delta-hedged option return is negatively related to the total volatility of the underlying stock, measured as the std.dev of daily stock returns." p.8 
- The VOL coefficient estimate in this situation in the paper is -0.0299; mine is -0.027797 (Table 2)
- "IVOL is in the paper defined as the standard deviation of the resioduals of the Fama-French 3-factors model estimated using the daily stock returns over the previous month" Table 1
- "Systematic Volatility is defined as sqrt(VOL^2 - IVOL^2)" p.9
- "All volatility measures are annualized" <- Wurzel-T-Gesetz reicht aus? Table 2

# V. Appendix

##### Descriptive statistics sorted by idiosyncratic volatility

In [18]:
sorted_low_first = final_dataset.sort_values(by=['idiosyncratic_volatility'], ascending=[True]).head(int(0.20*len(final_dataset)))
sorted_high_first = final_dataset.sort_values(by=['idiosyncratic_volatility'], ascending=[False]).head(int(0.20*len(final_dataset)))

In [19]:
mf.descriptive_statistics(sorted_low_first)

,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,44106.0,0.001940,0.030575,-6.519033e-01,-0.021838,-0.010744,-0.001423,0.009653,0.026408,0.821364
security_variance,44106.0,0.000622,0.000846,5.548191e-08,0.000179,0.000300,0.000452,0.000690,0.001093,0.042095
security_std_dev,44106.0,0.053452,0.023433,5.512594e-04,0.031334,0.040545,0.049748,0.061476,0.077366,0.480172
idiosyncratic_volatility,44106.0,0.000045,0.000023,6.917966e-11,0.000009,0.000027,0.000047,0.000065,0.000075,0.000082
systematic_volatility,44106.0,0.000577,0.000842,4.728263e-08,0.000156,0.000261,0.000400,0.000635,0.001035,0.042056
systematic_part_percent,44106.0,89.965090,5.567738,7.261000e+01,82.140000,85.560000,90.100000,94.260000,97.680000,100.000000


In [20]:
mf.descriptive_statistics(sorted_high_first)

,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,44106.0,-0.011114,0.107832,-4.659014,-0.094227,-0.047522,-0.013464,0.019810,0.066258,4.314043
security_variance,44106.0,0.014602,0.075856,0.002824,0.004906,0.006035,0.008517,0.013549,0.023071,11.827467
security_std_dev,44106.0,0.248440,0.135106,0.124361,0.163932,0.181805,0.215981,0.272419,0.355481,8.048708
idiosyncratic_volatility,44106.0,0.002422,0.013572,0.000744,0.000822,0.000964,0.001336,0.002154,0.003817,2.257320
systematic_volatility,44106.0,0.012180,0.062697,0.002057,0.004035,0.005016,0.007145,0.011396,0.019330,9.570148
systematic_part_percent,44106.0,83.480321,2.850736,72.620000,80.950000,81.320000,82.550000,84.780000,87.450000,98.870000


# Model 1

In [83]:
def calculate_volatilitys(option_data,stock_data):
    """
    Calculate volatilitys for each OptionID in the dataset.
    Return all results in a new table called "volatility_table".
    """
    stock_returns   = mf.transform_stock_returns(stock_data)
    factor_returns  = mf.transform_market_factors(stock_data)
    #create new columns for volatilitys
    option_data['maturity_minus_60_days']   = option_data['maturity'] - pd.DateOffset(days=60)
    option_data['maturity_minus_30_days']   = option_data['maturity'] - pd.DateOffset(days=30)
    option_data['security_variance']        = None
    option_data['security_std_dev']         = None
    option_data['idiosyncratic_volatility'] = None
    option_data['systematic_volatility']    = None
    option_data['systematic_part_percent']  = None
    option_data['stock_data_available']     = False

    for i in range(len(option_data)):
        security_id = option_data['SecurityID'].iloc[i]
        start_time  = option_data.iloc[i,5]
        end_time    = option_data.iloc[i,6]
        delta_days = (end_time - start_time).days
        
        try:
            # filter stock returns and factor returns (between -60 and -30 days before maturity of the option_id)
            filtered_stock_returns  = stock_returns[security_id][(stock_returns.index >= start_time) & (stock_returns.index <= end_time)]
            filtered_factor_returns = factor_returns.iloc[:,0:2][(factor_returns.index >= start_time) & (factor_returns.index <= end_time)]

            # if len(filtered_stock_returns) == 0: skip and go to next i
            if len(filtered_stock_returns) == 0:
                continue

            # calculate regression with stock and market returns to get idiosyncratic volatility
            y = filtered_stock_returns
            y = y.reset_index(drop=True)

            X = filtered_factor_returns # market returns
            X = X.reset_index(drop=True)
            X = sm.add_constant(X)

            model = sm.OLS(y,X)
            results = model.fit()

            option_data.iloc[i,7]       = np.var(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,8]       = np.std(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,9]       = results.resid.std()*np.sqrt(delta_days) # <-idiosyncratic volatility
            option_data.iloc[i,10]      = np.sqrt((option_data.iloc[i,8]**2 - option_data.iloc[i,9]**2))
            option_data.iloc[i,11]      = round(100*option_data.iloc[i,10]/option_data.iloc[i,8],2)
        except:
            option_data.iloc[i,12]      = True
    
    option_data['security_variance']        = option_data['security_variance'].astype(float)
    option_data['security_std_dev']         = option_data['security_std_dev'].astype(float)
    option_data['idiosyncratic_volatility'] = option_data['idiosyncratic_volatility'].astype(float)
    option_data['systematic_volatility']    = option_data['systematic_volatility'].astype(float)
    option_data['systematic_part_percent']  = option_data['systematic_part_percent'].astype(float)

    return option_data

In [ ]:
def final_regression(option_data,constant):
    """
    This function calculates the regression of the delta-gain-hedge (dependend variable) on the idiosyncratic volatility and the systematic volatility (independent variables).
    """
    if constant == True:
        y = option_data['delta_gain']
        y = y.reset_index(drop=True)

        X = option_data[['security_std_dev']] # market returns
        X = X.reset_index(drop=True)
        X = sm.add_constant(X)

        model = sm.OLS(y,X)
        results = model.fit()
        risk_premia = pd.DataFrame(results.params)
        print(results.summary())
        print(results.pvalues)
        print('')
        print("Beta factors with constant:")
    else:
        y = option_data['delta_gain']
        y = y.reset_index(drop=True)

        X = option_data[['security_std_dev']] # market returns
        X = X.reset_index(drop=True)
        #X = sm.add_constant(X)

        model = sm.OLS(y,X)
        results = model.fit()
        risk_premia = pd.DataFrame(results.params)
        print(results.summary())
        print(results.pvalues)
        print('')
        print("Beta factors without constant:")

    return risk_premia

In [84]:
final_dataset = delta_gain_returns.sort_values(by=['SecurityID','maturity'], ascending=[True,True])
final_dataset = calculate_volatilitys(final_dataset,stock_data)
final_dataset = final_dataset.dropna()
mf.final_regression(final_dataset,True)

Dimension after transofrmation:  (2509, 6269)
Dimension: (11323542, 4)
Dimension after dropping duplicates:  (2509, 4)
                            OLS Regression Results                            
Dep. Variable:             delta_gain   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     818.8
Date:                Tue, 22 Aug 2023   Prob (F-statistic):               0.00
Time:                        12:46:02   Log-Likelihood:             2.6648e+05
No. Observations:              202755   AIC:                        -5.329e+05
Df Residuals:                  202752   BIC:                        -5.329e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975

,0
const,0.006649
idiosyncratic_volatility,-0.073869
systematic_volatility,-0.001197


In [85]:
mf.descriptive_statistics(final_dataset)

,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,202755.0,-0.000608,0.065272,-4.659014e+00,-0.046121,-0.021408,-0.003904,0.014582,0.045136,4.314043
security_variance,202755.0,0.004054,0.020867,5.548191e-08,0.000446,0.000862,0.001852,0.004113,0.008439,4.757018
security_std_dev,202755.0,0.121387,0.086437,5.512594e-04,0.049445,0.068728,0.100708,0.150097,0.214993,5.104435
idiosyncratic_volatility,202755.0,0.097220,0.078730,0.000000e+00,0.034271,0.050771,0.078016,0.121688,0.181130,4.190132
systematic_volatility,202755.0,0.063424,0.050334,4.848842e-05,0.019638,0.032497,0.051740,0.080516,0.119576,5.104435
systematic_part_percent,202755.0,54.710948,22.611567,8.000000e-02,23.640000,37.880000,55.380000,71.710000,84.310000,100.000000


# Model 2

In [86]:
def calculate_volatilitys(option_data,stock_data):
    """
    Calculate volatilitys for each OptionID in the dataset.
    Return all results in a new table called "volatility_table".
    """
    stock_returns   = mf.transform_stock_returns(stock_data)
    factor_returns  = mf.transform_market_factors(stock_data)
    #create new columns for volatilitys
    option_data['maturity_minus_60_days']   = option_data['maturity'] - pd.DateOffset(days=60)
    option_data['maturity_minus_30_days']   = option_data['maturity'] - pd.DateOffset(days=30)
    option_data['security_variance']        = None
    option_data['security_std_dev']         = None
    option_data['idiosyncratic_volatility'] = None
    option_data['systematic_volatility']    = None
    option_data['systematic_part_percent']  = None
    option_data['stock_data_available']     = False

    for i in range(len(option_data)):
        security_id = option_data['SecurityID'].iloc[i]
        start_time  = option_data.iloc[i,5]
        end_time    = option_data.iloc[i,6]
        delta_days = (end_time - start_time).days
        
        try:
            # filter stock returns and factor returns (between -60 and -30 days before maturity of the option_id)
            filtered_stock_returns  = stock_returns[security_id][(stock_returns.index >= start_time) & (stock_returns.index <= end_time)]
            filtered_factor_returns = factor_returns.iloc[:,0][(factor_returns.index >= start_time) & (factor_returns.index <= end_time)]

            # if len(filtered_stock_returns) == 0: skip and go to next i
            if len(filtered_stock_returns) == 0:
                continue

            # calculate regression with stock and market returns to get idiosyncratic volatility
            y = filtered_stock_returns
            y = y.reset_index(drop=True)

            X = filtered_factor_returns # market returns
            X = X.reset_index(drop=True)
            X = sm.add_constant(X)

            model = sm.OLS(y,X)
            results = model.fit()

            option_data.iloc[i,7]       = np.var(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,8]       = np.std(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,9]       = results.resid.std()*np.sqrt(delta_days) # <-idiosyncratic volatility
            option_data.iloc[i,10]      = np.sqrt((option_data.iloc[i,8]**2 - option_data.iloc[i,9]**2))
            option_data.iloc[i,11]      = round(100*option_data.iloc[i,10]/option_data.iloc[i,8],2)
        except:
            option_data.iloc[i,12]      = True
    
    option_data['security_variance']        = option_data['security_variance'].astype(float)
    option_data['security_std_dev']         = option_data['security_std_dev'].astype(float)
    option_data['idiosyncratic_volatility'] = option_data['idiosyncratic_volatility'].astype(float)
    option_data['systematic_volatility']    = option_data['systematic_volatility'].astype(float)
    option_data['systematic_part_percent']  = option_data['systematic_part_percent'].astype(float)

    return option_data

In [87]:
final_dataset = delta_gain_returns.sort_values(by=['SecurityID','maturity'], ascending=[True,True])
final_dataset = calculate_volatilitys(final_dataset,stock_data)
final_dataset = final_dataset.dropna()
mf.final_regression(final_dataset,True)

Dimension after transofrmation:  (2509, 6269)
Dimension: (11323542, 4)
Dimension after dropping duplicates:  (2509, 4)
                            OLS Regression Results                            
Dep. Variable:             delta_gain   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     537.9
Date:                Tue, 22 Aug 2023   Prob (F-statistic):          1.19e-233
Time:                        12:49:49   Log-Likelihood:             2.4464e+05
No. Observations:              180221   AIC:                        -4.893e+05
Df Residuals:                  180218   BIC:                        -4.892e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975

,0
const,0.006078
idiosyncratic_volatility,-0.066845
systematic_volatility,0.002351


In [88]:
mf.descriptive_statistics(final_dataset)

,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,180221.0,-0.000256,0.062450,-4.659014e+00,-0.044101,-0.020790,-0.003765,0.014416,0.044302,4.314043
security_variance,180221.0,0.003676,0.017270,5.548191e-08,0.000446,0.000846,0.001783,0.003883,0.007862,4.757018
security_std_dev,180221.0,0.117829,0.079077,5.512594e-04,0.049421,0.068058,0.098812,0.145840,0.207517,5.104435
idiosyncratic_volatility,180221.0,0.096791,0.072663,4.555645e-05,0.035842,0.052144,0.078815,0.121066,0.178137,4.261854
systematic_volatility,180221.0,0.058110,0.045952,1.068624e-04,0.017286,0.029282,0.047246,0.073756,0.110148,4.095877
systematic_part_percent,180221.0,51.506765,22.860117,1.700000e-01,20.430000,33.930000,51.600000,68.900000,82.360000,100.000000


# Model 3

In [89]:
def calculate_volatilitys(option_data,stock_data):
    """
    Calculate volatilitys for each OptionID in the dataset.
    Return all results in a new table called "volatility_table".
    """
    stock_returns   = mf.transform_stock_returns(stock_data)
    factor_returns  = mf.transform_market_factors(stock_data)
    #create new columns for volatilitys
    option_data['maturity_minus_60_days']   = option_data['maturity'] - pd.DateOffset(days=60)
    option_data['maturity_minus_30_days']   = option_data['maturity'] - pd.DateOffset(days=30)
    option_data['security_variance']        = None
    option_data['security_std_dev']         = None
    option_data['idiosyncratic_volatility'] = None
    option_data['systematic_volatility']    = None
    option_data['systematic_part_percent']  = None
    option_data['stock_data_available']     = False

    for i in range(len(option_data)):
        security_id = option_data['SecurityID'].iloc[i]
        start_time  = option_data.iloc[i,5]
        end_time    = option_data.iloc[i,6]
        delta_days = (end_time - start_time).days
        
        try:
            # filter stock returns and factor returns (between -60 and -30 days before maturity of the option_id)
            filtered_stock_returns  = stock_returns[security_id][(stock_returns.index >= start_time) & (stock_returns.index <= end_time)]
            filtered_factor_returns = factor_returns.iloc[:,2][(factor_returns.index >= start_time) & (factor_returns.index <= end_time)]

            # if len(filtered_stock_returns) == 0: skip and go to next i
            if len(filtered_stock_returns) == 0:
                continue

            # calculate regression with stock and market returns to get idiosyncratic volatility
            y = filtered_stock_returns
            y = y.reset_index(drop=True)

            X = filtered_factor_returns # market returns
            X = X.reset_index(drop=True)
            X = sm.add_constant(X)

            model = sm.OLS(y,X)
            results = model.fit()

            option_data.iloc[i,7]       = np.var(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,8]       = np.std(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,9]       = results.resid.std()*np.sqrt(delta_days) # <-idiosyncratic volatility
            option_data.iloc[i,10]      = option_data.iloc[i,8] - option_data.iloc[i,9]
            option_data.iloc[i,11]      = round(100*option_data.iloc[i,10]/option_data.iloc[i,8],2)
        except:
            option_data.iloc[i,12]      = True
    
    option_data['security_variance']        = option_data['security_variance'].astype(float)
    option_data['security_std_dev']         = option_data['security_std_dev'].astype(float)
    option_data['idiosyncratic_volatility'] = option_data['idiosyncratic_volatility'].astype(float)
    option_data['systematic_volatility']    = option_data['systematic_volatility'].astype(float)
    option_data['systematic_part_percent']  = option_data['systematic_part_percent'].astype(float)

    return option_data

In [90]:
final_dataset = delta_gain_returns.sort_values(by=['SecurityID','maturity'], ascending=[True,True])
final_dataset = calculate_volatilitys(final_dataset,stock_data)
final_dataset = final_dataset.dropna()
mf.final_regression(final_dataset,True)

Dimension after transofrmation:  (2509, 6269)
Dimension: (11323542, 4)
Dimension after dropping duplicates:  (2509, 4)
                            OLS Regression Results                            
Dep. Variable:             delta_gain   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     884.6
Date:                Tue, 22 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:05:53   Log-Likelihood:             2.8382e+05
No. Observations:              220533   AIC:                        -5.676e+05
Df Residuals:                  220530   BIC:                        -5.676e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975

,0
const,0.006909
idiosyncratic_volatility,-0.065279
systematic_volatility,0.008827


In [91]:
mf.descriptive_statistics(final_dataset)

,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,220533.0,-0.000955,0.067076,-4.659014e+00,-0.047301,-0.021838,-0.004011,0.014595,0.045340,4.668170
security_variance,220533.0,0.004428,0.034340,5.548191e-08,0.000450,0.000876,0.001904,0.004282,0.008890,11.827467
security_std_dev,220533.0,0.124233,0.093915,5.512594e-04,0.049619,0.069280,0.102113,0.153147,0.220659,8.048708
idiosyncratic_volatility,220533.0,0.120914,0.091824,3.543446e-05,0.048085,0.067385,0.099362,0.148773,0.214618,7.871990
systematic_volatility,220533.0,0.003318,0.012507,-1.481687e-01,-0.002920,-0.001576,-0.000051,0.004347,0.013120,2.511444
systematic_part_percent,220533.0,2.656553,7.745696,-2.247000e+01,-2.240000,-1.830000,-0.060000,4.290000,11.050000,99.930000


# Model 4

In [92]:
def calculate_volatilitys(option_data,stock_data):
    """
    Calculate volatilitys for each OptionID in the dataset.
    Return all results in a new table called "volatility_table".
    """
    stock_returns   = mf.transform_stock_returns(stock_data)
    factor_returns  = mf.transform_market_factors(stock_data)
    #create new columns for volatilitys
    option_data['maturity_minus_60_days']   = option_data['maturity'] - pd.DateOffset(days=60)
    option_data['maturity_minus_30_days']   = option_data['maturity'] - pd.DateOffset(days=30)
    option_data['security_variance']        = None
    option_data['security_std_dev']         = None
    option_data['idiosyncratic_volatility'] = None
    option_data['systematic_volatility']    = None
    option_data['systematic_part_percent']  = None
    option_data['stock_data_available']     = False

    for i in range(len(option_data)):
        security_id = option_data['SecurityID'].iloc[i]
        start_time  = option_data.iloc[i,5]
        end_time    = option_data.iloc[i,6]
        delta_days = (end_time - start_time).days
        
        try:
            # filter stock returns and factor returns (between -60 and -30 days before maturity of the option_id)
            filtered_stock_returns  = stock_returns[security_id][(stock_returns.index >= start_time) & (stock_returns.index <= end_time)]
            filtered_factor_returns = factor_returns.iloc[:,0][(factor_returns.index >= start_time) & (factor_returns.index <= end_time)]

            # if len(filtered_stock_returns) == 0: skip and go to next i
            if len(filtered_stock_returns) == 0:
                continue

            # calculate regression with stock and market returns to get idiosyncratic volatility
            y = filtered_stock_returns
            y = y.reset_index(drop=True)

            X = filtered_factor_returns # market returns
            X = X.reset_index(drop=True)
            X = sm.add_constant(X)

            model = sm.OLS(y,X)
            results = model.fit()

            option_data.iloc[i,7]       = np.var(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,8]       = np.std(filtered_stock_returns)*np.sqrt(delta_days) #scaled with sqrt of observation days
            option_data.iloc[i,9]       = results.resid.std()*np.sqrt(delta_days) # <-idiosyncratic volatility
            option_data.iloc[i,10]      = option_data.iloc[i,8] - option_data.iloc[i,9]
            option_data.iloc[i,11]      = round(100*option_data.iloc[i,10]/option_data.iloc[i,8],2)
        except:
            option_data.iloc[i,12]      = True
    
    option_data['security_variance']        = option_data['security_variance'].astype(float)
    option_data['security_std_dev']         = option_data['security_std_dev'].astype(float)
    option_data['idiosyncratic_volatility'] = option_data['idiosyncratic_volatility'].astype(float)
    option_data['systematic_volatility']    = option_data['systematic_volatility'].astype(float)
    option_data['systematic_part_percent']  = option_data['systematic_part_percent'].astype(float)

    return option_data

In [93]:
final_dataset = delta_gain_returns.sort_values(by=['SecurityID','maturity'], ascending=[True,True])
final_dataset = calculate_volatilitys(final_dataset,stock_data)
final_dataset = final_dataset.dropna()
mf.final_regression(final_dataset,True)

Dimension after transofrmation:  (2509, 6269)
Dimension: (11323542, 4)
Dimension after dropping duplicates:  (2509, 4)
                            OLS Regression Results                            
Dep. Variable:             delta_gain   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     912.3
Date:                Tue, 22 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:09:41   Log-Likelihood:             2.8385e+05
No. Observations:              220533   AIC:                        -5.677e+05
Df Residuals:                  220530   BIC:                        -5.677e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975

,0
const,0.006323
idiosyncratic_volatility,-0.066120
systematic_volatility,-0.010102


In [94]:
mf.descriptive_statistics(final_dataset)

,count,mean,std,min,10%,25%,50%,75%,90%,max
delta_gain,220533.0,-0.000955,0.067076,-4.659014e+00,-0.047301,-0.021838,-0.004011,0.014595,0.045340,4.668170
security_variance,220533.0,0.004428,0.034340,5.548191e-08,0.000450,0.000876,0.001904,0.004282,0.008890,11.827467
security_std_dev,220533.0,0.124233,0.093915,5.512594e-04,0.049619,0.069280,0.102113,0.153147,0.220659,8.048708
idiosyncratic_volatility,220533.0,0.107507,0.092062,4.555645e-05,0.037645,0.055078,0.084962,0.133570,0.200169,8.229191
systematic_volatility,220533.0,0.016726,0.025313,-1.804828e-01,-0.001581,0.001670,0.009749,0.022956,0.041927,2.519988
systematic_part_percent,220533.0,15.424778,17.873929,-2.247000e+01,-1.580000,1.700000,10.210000,23.800000,39.918000,99.870000


# Model 5

In [95]:
def final_regression(option_data,constant):
    """
    This function calculates the regression of the delta-gain-hedge (dependend variable) on the idiosyncratic volatility and the systematic volatility (independent variables).
    """
    if constant == True:
        y = option_data['delta_gain']
        y = y.reset_index(drop=True)

        X = option_data[['security_std_dev']] # market returns
        X = X.reset_index(drop=True)
        X = sm.add_constant(X)

        model = sm.OLS(y,X)
        results = model.fit()
        risk_premia = pd.DataFrame(results.params)
        print(results.summary())
        print(results.pvalues)
        print('')
        print("Beta factors with constant:")
    else:
        y = option_data['delta_gain']
        y = y.reset_index(drop=True)

        X = option_data[['security_std_dev']] # market returns
        X = X.reset_index(drop=True)
        #X = sm.add_constant(X)

        model = sm.OLS(y,X)
        results = model.fit()
        risk_premia = pd.DataFrame(results.params)
        print(results.summary())
        print(results.pvalues)
        print('')
        print("Beta factors without constant:")

    return risk_premia

In [96]:
final_dataset = delta_gain_returns.sort_values(by=['SecurityID','maturity'], ascending=[True,True])

In [97]:
final_dataset = mf.calculate_volatilitys(final_dataset,stock_data)
final_dataset = final_dataset.dropna()
final_regression(final_dataset,True)

Dimension after transofrmation:  (2509, 6269)
Dimension: (11323542, 4)
Dimension after dropping duplicates:  (2509, 4)
                            OLS Regression Results                            
Dep. Variable:             delta_gain   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1729.
Date:                Tue, 22 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:39:52   Log-Likelihood:             2.8380e+05
No. Observations:              220533   AIC:                        -5.676e+05
Df Residuals:                  220531   BIC:                        -5.676e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------

,0
const,0.006871
security_std_dev,-0.062990
